In [6]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import welch
import os

# CHANGE DIRECTORY TO YOUR PROCESSED EEGS!!!!!
processed_eegs_dir = "/Users/estuardomelendez/Desktop/processed_eegs"
processed_files = [os.path.join(processed_eegs_dir, f) for f in os.listdir(processed_eegs_dir) if f.endswith(".parquet")]

# Sampling frequency (Hz) and window size (in seconds)
fs = 400  # Updated sampling frequency to match professor's files
window_size_seconds = 2  # Duration of each window in seconds
window_size_samples = fs * window_size_seconds

# Define EEG frequency bands
bands = {
    "delta": (1, 3),
    "theta": (4, 7),
    "alpha1": (8, 9),
    "alpha2": (10, 12),
    "beta1": (13, 17),
    "beta2": (18, 30),
    "gamma1": (31, 40),
    "gamma2": (41, 50),
    "higher": (51, 250)
}

# List of 19 standard EEG channels
expected_channels = [
    "Fp1", "Fp2", "F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2",
    "F7", "F8", "T3", "T4", "T5", "T6", "Fz", "Cz", "Pz"
]

# Function to split data into windows
def split_into_windows(signal, window_size):
    num_windows = len(signal) // window_size
    return [signal[i * window_size:(i + 1) * window_size] for i in range(num_windows)]

# Function to extract time-domain features
def extract_time_features(signal):
    return {
        "mean": np.mean(signal),
        "variance": np.var(signal),
        "skewness": skew(signal),
        "kurtosis": kurtosis(signal),
        "rms": np.sqrt(np.mean(signal**2)),
        "zero_crossing_rate": np.sum(np.diff(np.sign(signal)) != 0) / len(signal),
        "mean_abs": np.mean(np.abs(signal)),
        "diff_rms1": np.sqrt(np.mean(np.diff(signal) ** 2)),
        "diff_rms2": np.sqrt(np.mean(np.diff(signal, n=2) ** 2))
    }

# Function to extract frequency-domain features using FFT
def extract_frequency_features(signal, fs):
    L = len(signal)
    Y = np.fft.fft(signal)
    P2 = np.abs(Y / L)
    P1 = P2[:L // 2 + 1]
    P1[1:-1] *= 2
    freqs = fs * np.arange(L // 2 + 1) / L
    
    band_powers = {name: np.sum(P1[(freqs >= low) & (freqs <= high)]) for name, (low, high) in bands.items()}
    band_powers["spectral_entropy"] = -np.sum(P1 * np.log(P1 + 1e-10))
    return band_powers

# Function to extract features from a single EEG file
def extract_features_from_file(file_path, fs, window_size_samples):
    data = pd.read_parquet(file_path)
    available_channels = [ch for ch in expected_channels if ch in data.columns]
    
    if len(available_channels) < len(expected_channels):
        print(f"Warning: Some expected EEG channels are missing in {file_path}")
    
    all_features = []
    
    for channel in available_channels:
        signal = data[channel].values
        windows = split_into_windows(signal, window_size_samples)
        
        for i, window in enumerate(windows):
            time_features = extract_time_features(window)
            frequency_features = extract_frequency_features(window, fs)
            
            combined_features = {
                "file": os.path.basename(file_path),
                "channel": channel,
                "window": i,
                **time_features,
                **frequency_features
            }
            all_features.append(combined_features)
    
    return pd.DataFrame(all_features)

# Process all files
combined_features = []
for file_path in processed_files:
    print(f"Extracting features from: {file_path}")
    file_features = extract_features_from_file(file_path, fs, window_size_samples)
    combined_features.append(file_features)

# Combine features from all files into a single DataFrame
all_features_df = pd.concat(combined_features, ignore_index=True)

# Save extracted features to a CSV file
output_feature_file = "/Users/estuardomelendez/Desktop/processed_eegs/eeg_features.csv"
all_features_df.to_csv(output_feature_file, index=False)

# Preview extracted features
print(f"Features saved to: {output_feature_file}")
print("\nPreview of Extracted Features:")
print(all_features_df.head())


Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/4222573799.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/1622210937.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/678648250.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/2148256831.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/1469546440.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/3329232596.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/2718549792.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/2510219397.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/2436487142.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/2014107973.parquet
Extracting features from: /Users/estuardomelendez/Desktop/processed_eegs/10377733